In [1]:
import sys
import numpy as np

pi = np.loadtxt('hmmprior.txt')
pi = np.log(pi)

In [2]:
a = np.loadtxt('hmmtrans.txt')
a = np.log(a)

In [3]:
b = np.loadtxt('hmmemit.txt')
b = np.log(b)

In [4]:
f_tag = open("index_to_tag.txt",'r')
tag = f_tag.readlines()
tag_dict = {}
for i in range(len(tag)):
    tag_dict[tag[i].strip('\n')] = i+1

In [5]:
f_word = open("index_to_word.txt",'r')
word = f_word.readlines()
word_dict = {}
for i in range(len(word)):
    word_dict[word[i].strip('\n')] = i+1

In [6]:
f_input = open("testwords.txt",'r')
input = f_input.readlines()
traindata = []
for i in range(len(input)):
    sentence = input[i].strip('\n')
    word = sentence.split(' ')
    traindata.append(word)

splitdata = []
for i in range(len(traindata)): 
    sentence_line = []
    for j in range(len(traindata[i])):
        word_line = []
        splitword = traindata[i][j].split('_')[0]
        splittag = traindata[i][j].split('_')[1]
        word_line.append(splitword)
        word_line.append(splittag)
        sentence_line.append(word_line)
        
    splitdata.append(sentence_line)

for i in range(len(splitdata)):
    for j in range(len(splitdata[i])):
        splitdata[i][j][0] = word_dict.get(splitdata[i][j][0])
        splitdata[i][j][1] = tag_dict.get(splitdata[i][j][1])
print(splitdata)

[[[1218, 9], [35, 9], [1, 3], [1, 9], [1, 9], [720, 9], [3679, 9], [696, 9], [1, 9], [1, 9], [39, 9]], [[2423, 1], [291, 9]], [[3961, 2], [2122, 6], [4192, 9], [3080, 4], [3505, 8], [7759, 9], [5534, 9], [5507, 9], [435, 9], [6452, 9], [1801, 9], [4298, 9], [2476, 3], [4407, 9], [3543, 3], [4569, 9], [4222, 9], [5871, 9], [4227, 9], [439, 9], [7157, 9], [5817, 9], [7845, 9], [4953, 9], [7747, 9], [7641, 9], [6514, 9], [4315, 9], [7695, 9], [5690, 9], [6431, 9], [7695, 9], [4885, 9], [4651, 9], [39, 9]], [[3742, 9], [7503, 9], [6452, 9], [7760, 9], [34, 9], [7714, 9], [34, 9], [6202, 9], [4405, 9], [1, 9], [4298, 9], [7745, 9], [7126, 9], [1665, 3], [34, 9], [1523, 3], [4227, 9], [3640, 3], [4191, 9], [4709, 9], [5817, 9], [6452, 9], [7941, 9], [8044, 9], [2345, 3], [6141, 9], [7902, 9], [5507, 9], [6126, 9], [7739, 9], [5817, 9], [7697, 9], [1, 9], [6195, 9], [4159, 9], [2891, 3], [39, 9]], [[760, 9], [4508, 9], [3543, 3], [6504, 9], [5507, 9], [638, 9], [6452, 9], [7695, 9], [6462, 9]

In [7]:
def logsumexp(x,y):
    if x == float("-inf"):
        return y
    if y == float("-inf"):
        return x
    m = max(x,y)
    sum = m + np.log(1+np.exp(-abs(x-y)))
    return sum

In [8]:
def alpha_beta(data, tag_dict, a, b):
    # alpha of first sentence
    alpha = np.zeros((len(data), len(tag_dict)))
    xt = data[0][0]
    # first row of alpha
    alpha_n = np.add(b[:,xt - 1],pi)
    alpha[0] = alpha_n


    # start from the second word
    for i in range(1, len(data)):
        xi = data[i][0] - 1 
        for j in range(len(tag_dict)):
            sum = float("-inf")
            new_matrix = a[:,j] + alpha[i-1]
            for n in new_matrix:
                sum = logsumexp(sum,n)

            alpha[i][j] = sum + b[j][xi]

    beta = np.zeros((len(data), len(tag_dict)))

    for i in range(len(data)-1, 0, -1):
        xi = data[i][0] - 1
        for j in range(len(tag_dict)):
            sum = float("-inf")
            new_matrix = b[:,xi] + beta[i] + a[j]
            for n in new_matrix:
                sum = logsumexp(sum,n)
            beta[i-1][j] = sum

    #print(alpha)
    return alpha, beta

In [14]:
# predict
def predict(alpha, beta, data):
    predict_matrix = alpha + beta
    y_pre = np.argmax(predict_matrix, axis=1)+1
    y_pre_tag = []
    for index in y_pre:
        for k,v in tag_dict.items():
            if(v == index):
                y_pre_tag.append(k)

    word = []
    for i in range(len(data)):
        for k,v in word_dict.items():
            if(v == data[i][0]):
                word.append(k)

    output = ""
    for i in range(len(word)):
        output = output + word[i] + '_' + y_pre_tag[i] + " "
    output = output + '\n'
    
    return output, y_pre

In [15]:
def log_likelihood(alpha): 
    last_row = alpha[len(alpha)-1]
    likelihood = np.sum(last_row)
    return likelihood

In [17]:
ave_likelihood = 0
count = 0
num_word = 0
for data in splitdata:
    m,n = alpha_beta(data, tag_dict, a, b)
    out_write, tag_compare = predict(m,n,data)
    num_word += len(data)
    for i in range(len(data)):
        if data[i][1] != tag_compare[i]:
            count +=1
accuracy = count/num_word  
    
    #ave_likelihood = ave_likelihood + log_likelihood(m)
    
# ave_likelihood = ave_likelihood/len(splitdata)
# print(ave_likelihood)